In [ ]:
import re
import csv
import json
import emoji
import neologdn
import unicodedata
from pathlib import Path


# 絵文字の出現回数をカウントし、降順にソートする
def emoji_count_and_sort(dataset, sort=True, descending_order=True):

    # 絵文字の出現回数をカウントする
    emoji_count = {}
    for d in dataset:
        if d["label"] not in emoji_count:
            emoji_count[d["label"]] = 1
        else:
            emoji_count[d["label"]] = emoji_count[d["label"]] + 1

    # ソートする
    if sort:
        emoji_count = dict(sorted(emoji_count.items(), key=lambda x:x[1], reverse=descending_order))
        return emoji_count
    
    return emoji_count


# テキサスクリーニング
def clean_text(dataset, del_num=True):
    for i in range(0, len(dataset), 1):
        dataset[i]["text"] = re.sub(r'@[a-zA-Z0-9_]+', "", dataset[i]["text"])
        dataset[i]["text"] = re.sub(r'http[a-zA-Z0-9_:./]+', "", dataset[i]["text"])
        #dataset[i]["text"] = re.sub(r'[a-zA-Z_]', "", dataset[i]["text"])
        if del_num:
            dataset[i]["text"] = re.sub(r'[0-9]', "0", dataset[i]["text"])
        dataset[i]["text"] = neologdn.normalize(dataset[i]["text"])

    # 空の文字列の削除
    dataset = [d for d in dataset if d["text"]]
    
    return(dataset)


# 全生ツイートの取得関数
def get_tweets():

    # ツイートデータのパスを取得する
    p = Path('.')
    csv_paths = list(p.glob('./dataset/**/*.csv'))
    
    # 全 csv からツイートを読み込む
    tweets = []
    for csv_path in csv_paths:
        with open(csv_path, 'r', encoding='utf-8') as f:
            reader = csv.reader(f)
            texts_list = [row for row in reader]
        # 読み込んだ多次元リストを1次元リストに変換
        for tl in texts_list:
            for i in range(0, len(tl), 1):
                tweets.append(tl[i])
    print(f"全生ツイート: {len(tweets)}")
    #print(tweets[test_index])

    return tweets


# 絵文字が一種類しかないツイートの取得
def get_one_emoji_tweets(text_cleaning=True, del_num=True, emoji_position_end=True, emoji_version=15, text_len_min=0, text_len_max=500, create_json=False):

    # 全生ツイートの取得
    tweets = get_tweets()

    # NFKC正規化
    tweets = [unicodedata.normalize("NFKC", t) for t in tweets]
    #print(len(tweets))
    #print(tweets[test_index])

    # 絵文字が一種類しかないテキストの抽出
    one_emoji_tweets = [t for t in tweets if emoji.emoji_count(t, unique=True) == 1]
    print(f"\n\
絵文字が一種類しかない生ツイートの数: {len(one_emoji_tweets)}\
 (全生ツイートにおける割合: {(len(one_emoji_tweets) / len(tweets)) * 100}%)\
    ")
    #print(one_emoji_tweets[test_index])

    # emoji_version 以下の絵文字付きのツイートを取得する
    num_of_one_emoji_tweets = len(one_emoji_tweets)
    one_emoji_tweets = [t for t in one_emoji_tweets if emoji.EMOJI_DATA[emoji.distinct_emoji_list(t)[0]]["E"] <= emoji_version]
    print(f"\n\
Unicode Version {emoji_version:.1f} 以下の絵文字付きツイートの数： {len(one_emoji_tweets)}\
 (絵文字が一種類しかないツイートにおける割合： {(len(one_emoji_tweets) / num_of_one_emoji_tweets) * 100})\
          ")
    
    # 絵文字が末尾にあるツイートを取得する
    if emoji_position_end:
        one_emoji_tweets = [t for t in one_emoji_tweets if emoji.emoji_list(t)[0]["match_end"] == len(t)]
        print(f"\n\
絵文字が末尾にあるツイートの数: {len(one_emoji_tweets)}\
 (絵文字が一種類しかないツイートにおける割合: {(len(one_emoji_tweets) / num_of_one_emoji_tweets) * 100}%)\
    ")
        
    # 絵文字が一種類しかないツイートのデータセットの作成
    one_emoji_dataset = [
        {"text": emoji.replace_emoji(t, replace=""),"label": emoji.distinct_emoji_list(t)[0]} 
        for t in one_emoji_tweets
    ]
    #print(len(one_emoji_dataset))

    # テキストクリーニング
    if text_cleaning:
        one_emoji_dataset = clean_text(one_emoji_dataset, del_num=del_num)

    # テキストの長さを制限する
    one_emoji_dataset = [d for d in one_emoji_dataset if (len(d["text"]) >= text_len_min) and (len(d["text"]) <= text_len_max)]
    print(f"\n\
長さ制限: {text_len_min} ~ {text_len_max}\n\
長さ制限後のツイートの数: {len(one_emoji_dataset)}\
(絵文字が一種類しかないツイートにおける割合: {(len(one_emoji_dataset) / num_of_one_emoji_tweets) * 100}%)\
          ")

    # jsonに出力
    if create_json:
        json_path = "./dataset/one_emoji_dataset.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(one_emoji_dataset, f, indent=4, ensure_ascii=False)

    return one_emoji_dataset


# 出現回数上位 n 個の絵文字のツイートを取得する
def get_topn_one_emoji_tweets(topn=20, text_cleaning=True, del_num=True, qualified_diff=True, emoji_position_end=True, emoji_version=15, text_len_min=0, text_len_max=500, create_json=False):

    # 絵文字が一種類しかないツイートの取得
    one_emoji_dataset = get_one_emoji_tweets(text_cleaning=text_cleaning,
                                             del_num=del_num,
                                             emoji_position_end=emoji_position_end,
                                             emoji_version=emoji_version,
                                             text_len_min=text_len_min,
                                             text_len_max=text_len_max,
                                             create_json=create_json
                                             )

    # 絵文字出現回数の取得
    emoji_count = emoji_count_and_sort(one_emoji_dataset)
    print(f"\
\n全絵文字種類数（ステータスマージ前）: {len(emoji_count)}\
（世界中の全絵文字における割合：{(len(emoji_count) / 5025) * 100}）\
          ")

    # 各絵文字の絵文字が一種類しかないツイートにおける割合の取得
    emoji_count_temp = {}
    for e, n in emoji_count.items():
        emoji_count_temp[e] = [n, "{:.2f}".format((n / len(one_emoji_dataset)) * 100)]
    emoji_count = emoji_count_temp
    #print(len(one_emoji_dataset))
    print(f"全絵文字とその出現回数（ステータスマージ前）: {emoji_count}")

    # ステータス違いを無視する時
    if not qualified_diff:

        # ステータス違いがある絵文字の取得
        diff_status_emojis_list = []
        for e in emoji_count.keys():
            diff_status_emojis = [ec for ec in emoji_count.keys() if emoji.EMOJI_DATA[e]["en"] == emoji.EMOJI_DATA[ec]["en"]]
            if (diff_status_emojis not in diff_status_emojis_list) and (len(diff_status_emojis) != 1):
                diff_status_emojis_list.append(diff_status_emojis)
        print(f"\nステータス違いがある絵文字の種類数: {len(diff_status_emojis_list)}")
        print(f"ステータス違いがある絵文字: {diff_status_emojis_list}")

        # fully-qualified じゃない絵文字を fully-qualified 絵文字に変える
        for i in range(0, len(one_emoji_dataset), 1):
            for el in diff_status_emojis_list:
                if one_emoji_dataset[i]["label"] in el:
                    for e in el:
                        if emoji.EMOJI_DATA[e]["status"] == 2:
                            one_emoji_dataset[i]["label"] = e
        emoji_count = emoji_count_and_sort(one_emoji_dataset)
        print(f"\
\n全絵文字種類数（ステータスマージ後）: {len(emoji_count)}\
（世界中の全絵文字における割合：{(len(emoji_count) / 5025) * 100}、fully-qualifiedの全絵文字における割合：{(len(emoji_count) / 3773) * 100}）\
              ")

        # 各絵文字の絵文字が一種類しかないツイートにおける割合の取得
        emoji_count_temp = {}
        for e, n in emoji_count.items():
            emoji_count_temp[e] = [n, "{:.2f}".format((n / len(one_emoji_dataset)) * 100)]
        emoji_count = emoji_count_temp
        #print(len(one_emoji_dataset))
        print(f"全絵文字とその出現回数（ステータスマージ後）: {emoji_count}")

    # トップ n 個の絵文字の取得
    i = 0
    emoji_count_topn_total = 0
    emoji_count_topn_temp = {}
    emoji_count_topn = {}
    for e, ei in emoji_count.items():
        if i >= topn:
            break
        emoji_count_topn_temp[e] = ei
        emoji_count_topn_total = emoji_count_topn_total + ei[0]
        i = i + 1
    for e, ei in emoji_count_topn_temp.items():
        ei[1] = (ei[0] / emoji_count_topn_total) * 100
        ei[1] = "{:.2f}".format(ei[1])
        emoji_count_topn[e] = ei
    print(f"\n出現回数トップ {topn} 絵文字: {emoji_count_topn}")

    # トップ n 個の絵文字を含むデータセットの作成
    topn_one_emoji_dataset = [d for d in one_emoji_dataset if d["label"] in emoji_count_topn]
    print(f"出現回数トップ {topn} 絵文字を含むツイート数: {len(topn_one_emoji_dataset)}")

    # json に出力
    if create_json:
        json_path = f"./dataset/top{topn}_one_emoji_dataset.json"
        with open(json_path, 'w', encoding='utf-8') as f:
            json.dump(topn_one_emoji_dataset, f, indent=4, ensure_ascii=False)


def main():
    ############################################################################################################
    # パラメータ設定

    # 出現回数上位何位まで取り出す
    topn = 20
    # fully-qualified 絵文字にマージするかどうか、False はマージする
    qualified_diff = False
    # テキストクリーニングを行うかどうか
    text_cleaning = True
    # 数字を削除するかどうか
    del_num = False
    # 絵文字が末尾にあるツイートを取得するかどうか
    emoji_position_end = True
    # emoji_version 以下の絵文字付きのツイートの取得
    emoji_version = 13.1
    # テキスト長さ制限
    text_len_min = 0
    text_len_max = 5000
    # jsonファイルに出力するかどうか
    create_json = True
    ############################################################################################################
    
    get_topn_one_emoji_tweets(topn=topn,
                              qualified_diff=qualified_diff,
                              text_cleaning=text_cleaning,
                              del_num=del_num,
                              emoji_position_end=emoji_position_end,
                              emoji_version=emoji_version,
                              text_len_min=text_len_min,
                              text_len_max=text_len_max,
                              create_json=create_json
                              )


if __name__ == "__main__":
    main()